In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Importing relevant Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
%matplotlib inline
import requests
import io
import random
sns.set(style="ticks", color_codes=True)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import timeit
from tensorflow.keras.callbacks import EarlyStopping
from fbprophet import Prophet

Reading the data

In [ ]:
df = pd.read_csv('/kaggle/input/ece657aw20asg4coronavirus/time_series_covid19_deaths_global.csv')

In [ ]:
df.head()

In [ ]:
df_canada = df.loc[df['Country/Region']=='Canada'].iloc[:,4:]

In [ ]:
df_canada.shape

In [ ]:
df_canada.isnull().sum()

In [ ]:
death_daily = df_canada.sum(axis = 0)

In [ ]:
death_daily.head(-15)

In [ ]:
death_daily.index=pd.to_datetime(death_daily.index)
death_daily.head()

In [ ]:
plt.figure(figsize=(9,7))
plt.plot(death_daily)
plt.title("Daily Death in Canada")

In [ ]:
death_ca = death_daily.diff().fillna(death_daily[0]).astype(np.int)

In [ ]:
death_ca.head(-5)

In [ ]:
plt.figure(figsize=(9,7))
plt.plot(death_ca)
plt.title("Daily recovery")

In [ ]:
death_ca=pd.DataFrame(death_ca)
death_ca.index.name = 'Datetime'
death_ca
col=['Daily_death']
death_ca.columns=col
death_ca

In [ ]:
death_ca.shape

In [ ]:
split_date = '2020-04-05'
train = death_ca.loc[death_ca.index <= split_date].copy()
test = death_ca.loc[death_ca.index > split_date].copy()

In [ ]:
train.shape, test.shape

In [ ]:
test

In [ ]:
test \
    .rename(columns={'Daily_death': 'TEST SET'}) \
    .join(train.rename(columns={'Daily_death': 'TRAINING SET'}),
          how='outer') \
    .plot(figsize=(15,5), title='Death Daily', style='.')
plt.show()

In [ ]:
# Format data for prophet model using ds and y
train.reset_index() \
    .rename(columns={'Datetime':'ds',
                     'Daily_death':'y'}).head()

In [ ]:

# Setup and train model and fit
model = Prophet()
model.fit(train.reset_index() \
              .rename(columns={'Datetime':'ds',
                               'Daily_death':'y'}))

In [ ]:
test_forcast = model.predict(df=test.reset_index() \
                                   .rename(columns={'Datetime':'ds'}))

In [ ]:
test_forcast

In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
fig = model.plot(test_forcast,ax=ax)
plt.show()

In [ ]:
# Plot the components of the model
fig = model.plot_components(test_forcast)

In [ ]:
# Plot the forecast with the actuals
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(test.index, test['Daily_death'], color='r')
fig = model.plot(test_forcast, ax=ax)